In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import pandas as pd, numpy as np 
from travis_attack.config import Config
from travis_attack.insights import get_training_dfs, _prepare_df_concat
from travis_attack.utils import display_all
from IPython.core.debugger import set_trace
import logging 
logger = logging.getLogger("run")

In [ ]:
cfg = Config()
#run_name = "dashing-shadow-171"
run_name = "dummy-kYBrZsn7CxgZ2p6zygawVM"
path_run = f"{cfg.path_checkpoints}{run_name}/"

## Looking at some examples 

Below we just look at some examples to get a feel for what is going on. 

In [ ]:
def get_interesting_idx(df, n):
    def get_idx_with_top_column_values(cname, n=5, ascending=False):
        return df[['idx',cname]].\
            drop_duplicates().\
            sort_values(cname, ascending=ascending)\
            ['idx'][0:n].values.tolist()
    
    def sample_idx_with_label_flips(n=5): 
        df1 = df[['idx','label_flip']].query("label_flip!=0")
        if len(df1) == 0 : print("No label flips detected"); return None
        else: return df1.drop_duplicates()['idx'].sample(n).values.tolist()
    
    idx_d = dict(
        random = df.idx.drop_duplicates().sample(n).tolist(),
        label_flips = sample_idx_with_label_flips(n=n),
       #idx_n_unique_pp  = get_idx_with_top_column_values('idx_n_unique_pp',n=n,ascending=False),
       # idx_n_pp_changes = get_idx_with_top_column_values('idx_n_pp_changes',n=n,ascending=False),
       #high_contradiction = get_idx_with_top_column_values('contradiction_scores',n=n,ascending=False)
    )
    return idx_d

def print_stats(df, idx_d, key, i):
    print("\n###############\n")
    print(key, i+1, "\n")
    if idx_d[key] is None: return
    idx = idx_d[key][i]
    # Setup 
    df1 = df.query('idx==@idx')
    orig = pd.unique(df1['orig'])[0]
    print("Original:", orig)
    print("Original label", pd.unique(df1['label'])[0] )
    pp_all = list(df1['pp'])
    #print("All paraphrases", pp_all)
    pp_unique = list(pd.unique(df1['pp']))
    n_pp_unique = len(pp_unique)

    # showing a "timeline" of how the paraphrases change over the epochs
    g_fields = ["pp","reward_pp","reward_pp_minus_baseline", "vm_scores","acceptability_scores","sts_scores",
                "contradiction_scores", "pp_letter_diff",'pp_logp','ref_logp']
                #,
                
    #g_fields = ["pp","vm_scores"]
    g = df1.groupby(g_fields).agg({'epoch' : lambda x: list(x)})
    g = g.sort_values(by='epoch', key = lambda col: col.map(lambda x: np.min(x)))
    print("Unique paraphrases:", n_pp_unique)
    print("How the paraphrases change:")
    display_all(g)

    # Showing a dataframe of the few best paraphrases
    best_pps = df1.sort_values('reward_pp', ascending=False).iloc[0]
    print("Best Paraphrase")
    display_all(best_pps.to_frame().T)
        
def print_interesting_text_stats_training_step(df, n): 
    idx_d = get_interesting_idx(df, n)
    for key in idx_d.keys():
        for i in range(n): 
            print_stats(df, idx_d, key,i)

def show_random_examples_for_eval_set(df, n): 
    random_idx = df.idx.drop_duplicates().sample(n).tolist()
    cols = ['epoch','pp', 'is_adv_example', 'is_valid_pp', 'reward_pp', "vm_scores", 'acceptability_scores',
             'sts_scores', 'contradiction_scores',  'pp_letter_diff']#
         #   ,] #'edit_distance_token_level'
    
    metric_cols = [o for o in cols if o not in ['pp', 'epoch']]
    agg_d = {k:np.mean for k in metric_cols}
    for i, idx in enumerate(random_idx): 
        print("\n###############\n")
        print("random", i+1, "\n")
        df1 = df.query("idx==@idx")
        print("Original:", pd.unique(df1['orig'])[0])
        print("Original label", pd.unique(df1['label'])[0] )
        df2 = df1[cols]
        print("Paraphrase-level view")
        df3 = df2.groupby('pp').agg({'epoch': lambda x: list(x), **agg_d})
        df3.sort_values(by ='epoch', key=lambda col: col.map(lambda x: np.min(x)), inplace=True)
        display_all(df3)
        print("Epoch-level view")
        df4 = df2.groupby('epoch').agg(agg_d)
        df4.columns = [o + "_avg" for o in df4.columns]
        display_all(df4)
    #     print("Whole table")
    #     display_all(df2)
    return 


In [ ]:
split = 'valid'
n=1

df_d = get_training_dfs(path_run, postprocessed=False)
df = df_d[split]
if split == "training_step":
    idx_d = get_interesting_idx(df, n)
    print_interesting_text_stats_training_step(df, n)
else: 
    show_random_examples_for_eval_set(df, n)


###############

random 1 

Original: Operating profit improved to EUR 20.3 mn from EUR 11.4 mn .
Original label 2
Paraphrase-level view


,epoch,is_adv_example,is_valid_pp,reward_pp,vm_scores,acceptability_scores,sts_scores,contradiction_scores,pp_letter_diff
pp,,,,,,,,,
Operating profit rose from 11.4 million to 20.3 million.,"[0, 1, 2, 4]",0.0,0.0,0.000000,-3.314018e-05,0.554759,0.709238,0.031064,3.0
Operating profit rose to EUR 20.3 mn from EUR 11.4 mn..,[0],0.0,0.0,0.000000,-3.564358e-05,0.214589,0.854301,0.019030,4.0
Operating profit rose to 20.3 mn from 11.4 mn.,"[0, 1, 2, 3, 4]",0.0,0.0,0.000000,-3.123283e-05,0.420167,0.729731,0.022845,13.0
The company's operating profit rose from 11.4 million to 20.3 million.,[0],0.0,0.0,0.000000,-3.290176e-05,0.834865,0.680999,0.025087,-11.0
Operating profit rose to 20.3 million from 11.4 million.,"[0, 1]",0.0,0.0,0.000000,-3.409386e-05,0.583792,0.710019,0.017429,3.0
Operating profit rose from EUR 11.4 million to EUR 20.3 million.,"[0, 2, 4]",0.0,0.0,0.000000,-3.850460e-05,0.363996,0.782092,0.017329,-5.0
The company's operating profit rose to EUR 20.3 mn from EUR 11.4 mn.,"[0, 1]",0.0,0.0,0.000000,-3.802776e-05,0.319827,0.823476,0.007673,-9.0
Operating profit rose from 11.4 mn to 20.3 mn.,"[0, 1, 2, 3, 4]",0.0,0.0,0.000000,-2.181530e-05,0.339495,0.740000,0.024639,13.0
The operating profit rose to EUR 20.3 mn from EUR 11.4 mn.,"[0, 1, 2, 3, 4]",0.0,0.0,0.000000,-4.327297e-05,0.237987,0.858593,0.016522,1.0


,is_adv_example_avg,is_valid_pp_avg,reward_pp_avg,vm_scores_avg,acceptability_scores_avg,sts_scores_avg,contradiction_scores_avg,pp_letter_diff_avg
epoch,,,,,,,,
0,0.0,0.000000,0.000000e+00,-0.000029,0.445473,0.783311,0.018768,0.500000
1,0.0,0.035714,3.576279e-07,-0.000025,0.461045,0.778522,0.025432,-2.321429
2,0.0,0.038462,0.000000e+00,-0.000029,0.526953,0.783072,0.017475,-0.615385
3,0.0,0.041667,0.000000e+00,-0.000025,0.430960,0.803624,0.021026,-0.875000
4,0.0,0.058824,1.230661e-05,0.000073,0.513262,0.777161,0.023670,-0.147059


Epoch-level view


In [ ]:
df.sample(50)['orig'].tolist()

ValueError: Cannot take a larger sample than population when 'replace=False'

In [ ]:
%debug

> /home/tproth/Programs/miniconda/envs/nlp_env/lib/python3.8/site-packages/pandas/core/groupby/grouper.py(882)get_grouper()
    880                 in_axis, level, gpr = False, gpr, None
    881             else:
--> 882                 raise KeyError(gpr)
    883         elif isinstance(gpr, Grouper) and gpr.key is not None:
    884             # Add key to exclusions

--KeyboardInterrupt--
ipdb> q


## Looking at common removals and insertions 

In [ ]:
def get_common_removals_and_insertions(df_concat): 
    idx = df_concat[['data_split','orig_l', 'pp_l']].drop_duplicates().index
    df_unique_pp = df_concat[['data_split','orig_l', 'pp_l','insertions', 'removals']].iloc[idx]
    def flatten_list(l): return [item for sublist in l for item in sublist] 
    removals_flat   =  flatten_list(df_unique_pp['removals'].values)
    insertions_flat =  flatten_list(df_unique_pp['insertions'].values)
    return pd.value_counts(removals_flat), pd.value_counts(insertions_flat)

In [ ]:
df_concat = _prepare_df_concat(df_d)
removals, insertions = get_common_removals_and_insertions(df_concat)

print("\n#### REMOVALS ####\n")
print(removals.head(30))
print("\n#### INSERTIONS ####\n")
print(insertions.head(30))


#### REMOVALS ####

not                   7
this                  5
love                  3
film                  2
movie                 2
apple                 2
like                  1
hate                  1
I do not like this    1
this apple            1
do                    1
I do not like         1
I love                1
dtype: int64

#### INSERTIONS ####

.                                    21
n't                                   7
like                                  2
that                                  2
film.                                 2
- I like it                           1
thyme                                 1
really                                1
Not so much                           1
'.                                    1
:                                     1
''.                                   1
love                                  1
ie                                    1
cherry.                               1
enjoy                          

Here you can look at a specific phrase and examples of where it appears. 

In [ ]:
def investigate_phrase(phrase, cname, n ): 
    mask = [phrase in strs for strs in df_concat[cname]]
    display_all(df_concat[mask].sample(n))

In [ ]:
investigate_phrase('despite', 'removals', 4)

ValueError: a must be greater than 0 unless no samples are taken